# Business Understanding

## Business Objective

## Executive Summary

# Data Engineering

## Import Libraries

In [56]:
# import libraries required to load, transform, analyze and plot data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [6]:
# remove scientific notation and restrictions on df rows/columns display
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

## Data Dictionary

Data Source: IBM Cognos Telco Customer Churn Dataset

https://www.kaggle.com/yeanzc/telco-customer-churn-ibm-dataset

7043 observations with 33 variables

Column Definitions:

- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
- Country: The country of the customer’s primary residence.
- State: The state of the customer’s primary residence.
- City: The city of the customer’s primary residence.
- Zip Code: The zip code of the customer’s primary residence.
- Lat Long: The combined latitude and longitude of the customer’s primary residence.
- Latitude: The latitude of the customer’s primary residence.
- Longitude: The longitude of the customer’s primary residence.
- Gender: The customer’s gender: Male, Female
- Senior Citizen: Indicates if the customer is 65 or older: Yes, No
- Partner: Indicate if the customer has a partner: Yes, No
- Dependents: Indicates if the customer lives with any dependents: Yes, No. Dependents could be children, parents, grandparents, etc.
- Tenure Months: Indicates the total amount of months that the customer has been with the company by the end of the quarter specified above.
- Phone Service: Indicates if the customer subscribes to home phone service with the company: Yes, No
- Multiple Lines: Indicates if the customer subscribes to multiple telephone lines with the company: Yes, No
- Internet Service: Indicates if the customer subscribes to Internet service with the company: No, DSL, Fiber Optic, Cable.
- Online Security: Indicates if the customer subscribes to an additional online security service provided by the company: Yes, No
- Online Backup: Indicates if the customer subscribes to an additional online backup service provided by the company: Yes, No
- Device Protection: Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company: Yes, No
- Tech Support: Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times: Yes, No
- Streaming TV: Indicates if the customer uses their Internet service to stream television programing from a third party provider: Yes, No. The company does not charge an additional fee for this service.
- Streaming Movies: Indicates if the customer uses their Internet service to stream movies from a third party provider: Yes, No. The company does not charge an additional fee for this service.
- Contract: Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.
- Paperless Billing: Indicates if the customer has chosen paperless billing: Yes, No
- Payment Method: Indicates how the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check
- Monthly Charge: Indicates the customer’s current total monthly charge for all their services from the company.
- Total Charges: Indicates the customer’s total charges, calculated to the end of the quarter specified above.
- Churn Label: Yes = the customer left the company this quarter. No = the customer remained with the company. Directly related to Churn Value.
- Churn Value: 1 = the customer left the company this quarter. 0 = the customer remained with the company. Directly related to Churn Label.
- Churn Score: A value from 0-100 that is calculated using the predictive tool IBM SPSS Modeler. The model incorporates multiple factors known to cause churn. The higher the score, the more likely the customer will churn.
- CLTV: Customer Lifetime Value. A predicted CLTV is calculated using corporate formulas and existing data. The higher the value, the more valuable the customer. High value customers should be monitored for churn.
- Churn Reason: A customer’s specific reason for leaving the company. Directly related to Churn Category.

## Dataframe Basics

### Load File

In [61]:
# load primary source file to df
tc = pd.read_excel('data/Telco_customer_churn.xlsx', sheet_name='Telco_Churn')
tc.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.96,-118.27,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.06,-118.31,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.05,-118.29,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.06,-118.32,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,"3,046.05",Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.04,-118.27,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,"5,036.30",Yes,1,89,5340,Competitor had better devices


In [62]:
# drop columns not required or duplicative
drop_cols = ['Latitude', 'Longitude', 'Churn Label']
tc.drop(drop_cols, axis=1, inplace=True)

In [63]:
# convert column names to lower case without underscore
tc.columns = tc.columns.str.lower().str.replace(' ','_')

### Info

In [64]:
tc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customerid         7043 non-null   object 
 1   count              7043 non-null   int64  
 2   country            7043 non-null   object 
 3   state              7043 non-null   object 
 4   city               7043 non-null   object 
 5   zip_code           7043 non-null   int64  
 6   lat_long           7043 non-null   object 
 7   gender             7043 non-null   object 
 8   senior_citizen     7043 non-null   object 
 9   partner            7043 non-null   object 
 10  dependents         7043 non-null   object 
 11  tenure_months      7043 non-null   int64  
 12  phone_service      7043 non-null   object 
 13  multiple_lines     7043 non-null   object 
 14  internet_service   7043 non-null   object 
 15  online_security    7043 non-null   object 
 16  online_backup      7043 

**Notes on Info**
- Only non-null issue on Churn Reason (26.5% churned)
- Unexpected Data Types: Total Charges (float)
- Will convert indicator columns from text to 0/1

In [65]:
# check for duplicate customers
dups = tc[tc.duplicated(subset=['customerid'])==True]
print(f'{dups.shape[0]} duplicate CustomerIDs found')

0 duplicate CustomerIDs found


### Data Cleaning

In [66]:
# check value counts by column
col_list = ['gender', 'senior_citizen', 'partner', 'dependents', 
            'phone_service', 'multiple_lines', 'internet_service', 
            'online_security', 'online_backup', 'device_protection', 
            'tech_support', 'streaming_tv', 'streaming_movies', 'contract', 
            'paperless_billing', 'payment_method', 'churn_value', 
            'churn_reason', 'country', 'state', 'city', 'zip_code']

for col in col_list:
     print(f'\nValue Counts | column = {col}')
     print(tc[col].value_counts(normalize=True, dropna=False))


Value Counts | column = gender
Male     0.50
Female   0.50
Name: gender, dtype: float64

Value Counts | column = senior_citizen
No    0.84
Yes   0.16
Name: senior_citizen, dtype: float64

Value Counts | column = partner
No    0.52
Yes   0.48
Name: partner, dtype: float64

Value Counts | column = dependents
No    0.77
Yes   0.23
Name: dependents, dtype: float64

Value Counts | column = phone_service
Yes   0.90
No    0.10
Name: phone_service, dtype: float64

Value Counts | column = multiple_lines
No                 0.48
Yes                0.42
No phone service   0.10
Name: multiple_lines, dtype: float64

Value Counts | column = internet_service
Fiber optic   0.44
DSL           0.34
No            0.22
Name: internet_service, dtype: float64

Value Counts | column = online_security
No                    0.50
Yes                   0.29
No internet service   0.22
Name: online_security, dtype: float64

Value Counts | column = online_backup
No                    0.44
Yes                   0.34

**Notes on Value Counts**
- Change encoding to 0/1: Senior Citizen, Partner, Dependents, Phone Service, Multiple Lines, Internet Service, Online Security, Online Backup, Device Protection, Tech Support, Streaming TV, Streaming Movies, Paperless Billing
- Create Group: Churn Reason (Customer Service, Product/Network, Pricing, Moved, Unknown)
- One-Hot Coding: Gender, Contract, Payment Method, Churn Reason Group
- All customers in California, USA 
- City distribution is wide, < 10% in major cities. Likely rural telecom co. Explore other geographic attributes.
- Data values look clean, no corrupt values or other NaNs to deal with.

In [67]:
# replace yes-no variables with 0/1 for modeling
bool_list = ['senior_citizen', 'partner', 'dependents', 'phone_service', 
             'multiple_lines', 'internet_service', 'online_security', 
             'online_backup', 'device_protection', 'tech_support', 
             'streaming_tv', 'streaming_movies', 'paperless_billing']

for col in bool_list:
    tc.replace({col : { 'Yes' : 1, 'No' : 0, 'No phone service' : 0, 
                             'No internet service' : 0}}, inplace=True)

In [68]:
# create churn reason group to simplify
reason_map = {'Attitude of support person' : 'Cust Service',
              'Attitude of service provider' : 'Cust Service', 
              'Service dissatisfaction' : 'Cust Service', 
              'Lack of self-service on Website' : 'Cust Service', 
              'Poor expertise of phone support'  : 'Cust Service', 
              'Poor expertise of online support' : 'Cust Service', 
              'Competitor offered higher download speeds' : 'Product', 
              'Competitor offered more data' : 'Product', 
              'Competitor had better devices' : 'Product', 
              'Network reliability' : 'Product', 
              'Product dissatisfaction' : 'Product', 
              'Limited range of services' : 'Product', 
              'Competitor made better offer' : 'Pricing', 
              'Price too high' : 'Pricing', 
              'Extra data charges' : 'Pricing', 
              'Long distance charges' : 'Pricing', 
              'Lack of affordable download/upload speed' : 'Pricing', 
              'Moved' : 'Moved', 
              'Deceased' : 'Moved',
              'Don\'t know' : 'Unknown'
             }

tc.insert(loc=30, column='churn_rsn_grp', value=tc.churn_reason.map(reason_map))

In [77]:
# create list of multi-class variables for one-hot encoding
categoricals = ['gender', 'internet_service', 'contract', 'payment_method', 
                'churn_rsn_grp']

# create one-hot encoded dummy variables for categoricals
tc_ohe = pd.get_dummies(tc[categoricals], drop_first=True)
tc_ohe.rename(columns = {'gender_Male' : 'gender_male',
                         'internet_service_DSL' : 'internet_svc_dsl', 
                         'internet_service_Fiber optic' : 'internet_svc_fiber', 
                         'contract_One year' : 'contract_1yr', 
                         'contract_Two year' : 'contract_2yr', 
                         'payment_method_Credit card (automatic)' : 'pmt_meth_cc_auto', 
                         'payment_method_Electronic check' : 'pmt_meth_echeck', 
                         'payment_method_Mailed check' : 'pmt_meth_mail_check', 
                         'churn_rsn_grp_Moved' : 'churn_rsn_grp_moved', 
                         'churn_rsn_grp_Pricing' : 'churn_rsn_grp_pricing', 
                         'churn_rsn_grp_Product' : 'churn_rsn_grp_product', 
                         'churn_rsn_grp_Unknown' : 'churn_rsn_grp_unknown'
                        }, inplace = True)
tc_ohe.head()

,gender_male,internet_svc_dsl,internet_svc_fiber,contract_1yr,contract_2yr,pmt_meth_cc_auto,pmt_meth_echeck,pmt_meth_mail_check,churn_rsn_grp_moved,churn_rsn_grp_pricing,churn_rsn_grp_product,churn_rsn_grp_unknown
0,1,1,0,0,0,0,0,1,0,1,0,0
1,0,0,1,0,0,0,1,0,1,0,0,0
2,0,0,1,0,0,0,1,0,1,0,0,0
3,0,0,1,0,0,0,1,0,1,0,0,0
4,1,0,1,0,0,0,0,0,0,0,1,0


In [78]:
# concatenate OHE columns with original dataframe, and drop some columns not needed
tc_comb = pd.concat([tc, tc_ohe], axis=1)
tc_comb.drop(categoricals, axis=1, inplace=True)
tc_comb.head()

,customerid,count,country,state,city,zip_code,lat_long,senior_citizen,partner,dependents,tenure_months,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn_value,churn_score,cltv,churn_reason,gender_male,internet_svc_dsl,internet_svc_fiber,contract_1yr,contract_2yr,pmt_meth_cc_auto,pmt_meth_echeck,pmt_meth_mail_check,churn_rsn_grp_moved,churn_rsn_grp_pricing,churn_rsn_grp_product,churn_rsn_grp_unknown
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",0,0,0,2,1,0,1,1,0,0,0,0,1,53.85,108.15,1,86,3239,Competitor made better offer,1,1,0,0,0,0,0,1,0,1,0,0
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",0,0,1,2,1,0,0,0,0,0,0,0,1,70.70,151.65,1,67,2701,Moved,0,0,1,0,0,0,1,0,1,0,0,0
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",0,0,1,8,1,1,0,0,1,0,1,1,1,99.65,820.50,1,86,5372,Moved,0,0,1,0,0,0,1,0,1,0,0,0
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",0,1,1,28,1,1,0,0,1,1,1,1,1,104.80,"3,046.05",1,84,5003,Moved,0,0,1,0,0,0,1,0,1,0,0,0
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",0,0,1,49,1,1,0,1,1,0,1,1,1,103.70,"5,036.30",1,89,5340,Competitor had better devices,1,0,1,0,0,0,0,0,0,0,1,0


### Data Type Conversions

In [53]:
tc.head()

,customerid,count,country,state,city,zip_code,lat_long,gender,senior_citizen,partner,dependents,tenure_months,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn_value,churn_score,cltv,churn_reason,churn_reason_grp
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,86,3239,Competitor made better offer,Pricing
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",Female,0,0,1,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,67,2701,Moved,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",Female,0,0,1,8,1,1,Fiber optic,0,0,1,0,1,1,Month-to-month,1,Electronic check,99.65,820.50,1,86,5372,Moved,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",Female,0,1,1,28,1,1,Fiber optic,0,0,1,1,1,1,Month-to-month,1,Electronic check,104.80,"3,046.05",1,84,5003,Moved,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",Male,0,0,1,49,1,1,Fiber optic,0,1,1,0,1,1,Month-to-month,1,Bank transfer (automatic),103.70,"5,036.30",1,89,5340,Competitor had better devices,Product/Network


In [57]:
tc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customerid         7043 non-null   object 
 1   count              7043 non-null   int64  
 2   country            7043 non-null   object 
 3   state              7043 non-null   object 
 4   city               7043 non-null   object 
 5   zip_code           7043 non-null   int64  
 6   lat_long           7043 non-null   object 
 7   gender             7043 non-null   object 
 8   senior_citizen     7043 non-null   int64  
 9   partner            7043 non-null   int64  
 10  dependents         7043 non-null   int64  
 11  tenure_months      7043 non-null   int64  
 12  phone_service      7043 non-null   int64  
 13  multiple_lines     7043 non-null   int64  
 14  internet_service   7043 non-null   object 
 15  online_security    7043 non-null   int64  
 16  online_backup      7043 